<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/(%ED%86%B5%ED%95%A93)_kobert_nsmc3_mlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요 라이브러리 설치

In [1]:
!pip install transformers datasets accelerate scikit-learn optuna sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install transformers

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch
import optuna
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re
import platform
import transformers
import joblib

In [4]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


# 80% 데이터셋

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   nmsc3_0.5_aug_mlm.csv   이전   졸업논문


In [7]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/NSMC3_0.8_aug_mlm_final.csv', encoding='utf-8-sig')

In [8]:
df['label'].value_counts()

,count
label,
1,5000
0,5000


In [9]:
df_test=pd.read_excel('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_test.xlsx')

In [10]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

In [11]:
def clean_korean_documents(documents):
    # 텍스트 정제 (특수기호만 제거)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\sㄱ-ㅣ가-힣]', '', document)  # 특수기호만 제거, 정규 표현식
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# 한글이 아닌 문자를 제거하는 함수
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', text)

In [12]:
train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)


Cleaning Special Characters: 100%|██████████| 2000/2000 [00:00<00:00, 13024.09it/s]


In [13]:
# 3. 학습 데이터와 테스트 데이터에서 필요한 열 추출 및 라벨 인코딩
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. 라벨 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

In [14]:
# 5. KoBERT 모델을 위한 토크나이저 및 데이터셋 생성
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

tokenization_kobert.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [15]:
# 텍스트를 토큰화하는 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

In [16]:
# 데이터셋 생성 및 토크나이즈
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [17]:
# PyTorch 텐서 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT 모델 설정 및 학습 파라미터 최적화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
#CPU 측정

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU 사용량 및 속도 측정 함수
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # 시작 시간 기록
    start_cpu = psutil.cpu_percent(interval=None)  # 시작 CPU 사용량

    train_function()  # 학습 함수 호출

    end_time = time.time()  # 종료 시간 기록
    end_cpu = psutil.cpu_percent(interval=None)  # 종료 CPU 사용량

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [19]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

In [20]:
training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # 에포크 수 고정
    per_device_train_batch_size=64,
    learning_rate=5.0e-05,  # 최적화된 학습률
    weight_decay=0.01,  # 최적화된 weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",  # AdamW fused 적용 (PyTorch 2.0 이상)
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # 가장 좋은 모델을 accuracy 기준으로 선택
    greater_is_better=True,  # accuracy가 높은 모델을 저장
    save_total_limit=2,  # 저장할 모델 개수 제한
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
# 최적의 파라미터로 Trainer 구성
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 전체 학습 데이터로 평가
    compute_metrics=compute_metrics
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# CPU 사용량 및 속도 측정
measure_cpu_usage_and_time(trainer7.train)

Epoch,Training Loss,Validation Loss,F1
1,0.329300,0.396186,0.839923
2,0.197900,0.437026,0.827464
3,0.149600,0.565225,0.841277
4,0.062300,0.580774,0.849499
5,0.039000,0.665003,0.831509
6,0.016900,0.953728,0.827508
7,0.009600,0.830579,0.838808
8,0.002600,0.944906,0.836842
9,0.002900,0.946942,0.833010
10,0.001700,0.949093,0.838968


Elapsed Time: 1098.08 seconds
Average CPU Usage: 15.95%


In [23]:
# 모델 저장
#joblib.dump(trainer7, '/content/gdrive/MyDrive/NSMC2_0.8_original.pkl')

#print("모델이 저장되었습니다.")

In [24]:
# 8. 저장된 모델로 테스트 데이터 예측 및 결과 저장
# 저장된 모델 불러오기

#loaded_model = joblib.load('/content/gdrive/MyDrive/NSMC2_0.8_original.pkl')

# 불러온 모델을 사용하여 예측 수행
predictions7 = trainer7.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

In [25]:
test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]
selected_columns7.head(10)

,text,category_prediction
0,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,폴리스스토리 시리즈는 뉴까지 버릴께 하나도 없음 최고,0
3,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
5,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,완전 감동입니다 다시봐도 감동,1
7,개들의 나오나요 나오면 보고 싶음,1
8,내 나이와 같은 영화를 지금 본 나는 감동적이다하지만 훗날 다시보면대사하나하나 그 ...,1
9,영화의 오페라화라고 해야할 작품 극단적 평갈림은 어쩔 수 없는 듯,0


In [26]:
a=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [27]:
selected_columns7['y_true']=pd.DataFrame(a)
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7.to_csv('/content/nsmc3_0.8_mlm_prediction.csv', encoding='utf-8-sig')

In [28]:
from sklearn.metrics import f1_score
import pandas as pd
df=pd.read_csv('/content/nsmc3_0.8_mlm_prediction.csv', encoding='utf-8-sig')

from sklearn.metrics import classification_report, f1_score

# y_true와 y_pred를 추출
y_true = df['y_true']
y_pred = df['y_pred']

# classification_report를 사용하여 각 클래스별 F1-score 출력
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.8513    0.8470    0.8491      1000
           1     0.8478    0.8520    0.8499      1000

    accuracy                         0.8495      2000
   macro avg     0.8495    0.8495    0.8495      2000
weighted avg     0.8495    0.8495    0.8495      2000



# 50% 데이터셋

In [29]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/NSMC3_0.5_aug_mlm_final.csv', encoding='utf-8-sig')

In [30]:
df_test=pd.read_excel('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_test.xlsx')

In [31]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

def clean_korean_documents(documents):
    # 텍스트 정제 (특수기호만 제거)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\sㄱ-ㅣ가-힣]', '', document)  # 특수기호만 제거, 정규 표현식
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# 한글이 아닌 문자를 제거하는 함수
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', text)


train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)

# 3. 학습 데이터와 테스트 데이터에서 필요한 열 추출 및 라벨 인코딩
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. 라벨 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

# 5. KoBERT 모델을 위한 토크나이저 및 데이터셋 생성
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 텍스트를 토큰화하는 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

# 데이터셋 생성 및 토크나이즈
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# PyTorch 텐서 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT 모델 설정 및 학습 파라미터 최적화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)


Cleaning Special Characters: 100%|██████████| 2000/2000 [00:00<00:00, 13036.64it/s]


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
#CPU 측정

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU 사용량 및 속도 측정 함수
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # 시작 시간 기록
    start_cpu = psutil.cpu_percent(interval=None)  # 시작 CPU 사용량

    train_function()  # 학습 함수 호출

    end_time = time.time()  # 종료 시간 기록
    end_cpu = psutil.cpu_percent(interval=None)  # 종료 CPU 사용량

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [33]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # 에포크 수 고정
    per_device_train_batch_size=64,
    learning_rate=5.0e-05,  # 최적화된 학습률
    weight_decay=0.01,  # 최적화된 weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",  # AdamW fused 적용 (PyTorch 2.0 이상)
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # 가장 좋은 모델을 accuracy 기준으로 선택
    greater_is_better=True,  # accuracy가 높은 모델을 저장
    save_total_limit=2,  # 저장할 모델 개수 제한
    report_to="none"
)

# 최적의 파라미터로 Trainer 구성
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 전체 학습 데이터로 평가
    compute_metrics=compute_metrics
)

# CPU 사용량 및 속도 측정
measure_cpu_usage_and_time(trainer7.train)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.221500,0.446245,0.804832
2,0.185000,0.562629,0.814960
3,0.120700,0.459702,0.820413
4,0.039300,0.690011,0.819404
5,0.032400,0.788693,0.817874
6,0.019700,0.839806,0.835145
7,0.024300,0.860110,0.831885
8,0.009800,1.054495,0.821354
9,0.006900,1.137485,0.813446
10,0.002800,1.137737,0.816173


Elapsed Time: 1103.42 seconds
Average CPU Usage: 15.40%


In [34]:
# 모델 저장
#joblib.dump(trainer7, '/content/gdrive/MyDrive/NSMC2_0.5_original.pkl')

#print("모델이 저장되었습니다.")

In [35]:
# 8. 저장된 모델로 테스트 데이터 예측 및 결과 저장
# 저장된 모델 불러오기

#loaded_model = joblib.load('/content/gdrive/MyDrive/NSMC2_0.5_original.pkl')

# 불러온 모델을 사용하여 예측 수행
predictions7 = trainer7.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]

a=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

selected_columns7['y_true']=pd.DataFrame(a)
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7.to_csv('/content/nsmc3_0.5_mlm_prediction.csv', encoding='utf-8-sig')



In [36]:
from sklearn.metrics import f1_score
import pandas as pd

df=pd.read_csv('/content/nsmc3_0.5_mlm_prediction.csv', encoding='utf-8-sig')

from sklearn.metrics import classification_report, f1_score

# y_true와 y_pred를 추출
y_true = df['y_true']
y_pred = df['y_pred']

# classification_report를 사용하여 각 클래스별 F1-score 출력
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.8925    0.7640    0.8233      1000
           1     0.7937    0.9080    0.8470      1000

    accuracy                         0.8360      2000
   macro avg     0.8431    0.8360    0.8351      2000
weighted avg     0.8431    0.8360    0.8351      2000



# 10% 데이터셋

In [37]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/NSMC3_0.1_aug_mlm_final.csv', encoding='utf-8-sig')

df_test=pd.read_excel('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_test.xlsx')

In [38]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

def clean_korean_documents(documents):
    # 텍스트 정제 (특수기호만 제거)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\sㄱ-ㅣ가-힣]', '', document)  # 특수기호만 제거, 정규 표현식
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# 한글이 아닌 문자를 제거하는 함수
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', text)


train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)

# 3. 학습 데이터와 테스트 데이터에서 필요한 열 추출 및 라벨 인코딩
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. 라벨 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

# 5. KoBERT 모델을 위한 토크나이저 및 데이터셋 생성
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 텍스트를 토큰화하는 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

# 데이터셋 생성 및 토크나이즈
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# PyTorch 텐서 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT 모델 설정 및 학습 파라미터 최적화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)


Cleaning Special Characters: 100%|██████████| 2000/2000 [00:00<00:00, 13106.69it/s]


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
#CPU 측정

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU 사용량 및 속도 측정 함수
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # 시작 시간 기록
    start_cpu = psutil.cpu_percent(interval=None)  # 시작 CPU 사용량

    train_function()  # 학습 함수 호출

    end_time = time.time()  # 종료 시간 기록
    end_cpu = psutil.cpu_percent(interval=None)  # 종료 CPU 사용량

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [40]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # 에포크 수 고정
    per_device_train_batch_size=64,
    learning_rate=5.0e-05,  # 최적화된 학습률
    weight_decay=0.01,  # 최적화된 weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",  # AdamW fused 적용 (PyTorch 2.0 이상)
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # 가장 좋은 모델을 accuracy 기준으로 선택
    greater_is_better=True,  # accuracy가 높은 모델을 저장
    save_total_limit=2,  # 저장할 모델 개수 제한
    report_to="none"
)

# 최적의 파라미터로 Trainer 구성
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 전체 학습 데이터로 평가
    compute_metrics=compute_metrics
)

# CPU 사용량 및 속도 측정
measure_cpu_usage_and_time(trainer7.train)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.172200,0.745269,0.408736
2,0.111700,0.826526,0.762103
3,0.098800,0.809302,0.691389
4,0.063300,1.030320,0.689804
5,0.038400,1.750589,0.609343
6,0.043100,1.301686,0.592585
7,0.012800,1.497367,0.675776
8,0.022800,1.835609,0.646185
9,0.012700,1.881935,0.625087
10,0.013700,1.897935,0.639461


Elapsed Time: 1101.66 seconds
Average CPU Usage: 16.00%


In [41]:
# 모델 저장
#joblib.dump(trainer7, '/content/gdrive/MyDrive/NSMC2_0.1_original.pkl')

#print("모델이 저장되었습니다.")

In [42]:
# 8. 저장된 모델로 테스트 데이터 예측 및 결과 저장
# 저장된 모델 불러오기

#loaded_model = joblib.load('/content/gdrive/MyDrive/NSMC2_0.1_original.pkl')

# 불러온 모델을 사용하여 예측 수행
predictions7 = trainer7.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]

a=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

selected_columns7['y_true']=pd.DataFrame(a)
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7.to_csv('/content/nsmc3_0.1_mlm_prediction.csv', encoding='utf-8-sig')



In [43]:
from sklearn.metrics import f1_score
import pandas as pd

df=pd.read_csv('/content/nsmc3_0.1_mlm_prediction.csv', encoding='utf-8-sig')

from sklearn.metrics import classification_report, f1_score

# y_true와 y_pred를 추출
y_true = df['y_true']
y_pred = df['y_pred']

# classification_report를 사용하여 각 클래스별 F1-score 출력
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.8489    0.6460    0.7337      1000
           1     0.7143    0.8850    0.7905      1000

    accuracy                         0.7655      2000
   macro avg     0.7816    0.7655    0.7621      2000
weighted avg     0.7816    0.7655    0.7621      2000



# 2% 데이터셋

In [44]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/NSMC3_0.02_aug_mlm_final.csv', encoding='utf-8-sig')

df_test=pd.read_excel('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_test.xlsx')

In [45]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

def clean_korean_documents(documents):
    # 텍스트 정제 (특수기호만 제거)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\sㄱ-ㅣ가-힣]', '', document)  # 특수기호만 제거, 정규 표현식
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# 한글이 아닌 문자를 제거하는 함수
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', text)


train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)

# 3. 학습 데이터와 테스트 데이터에서 필요한 열 추출 및 라벨 인코딩
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. 라벨 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

# 5. KoBERT 모델을 위한 토크나이저 및 데이터셋 생성
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 텍스트를 토큰화하는 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

# 데이터셋 생성 및 토크나이즈
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# PyTorch 텐서 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT 모델 설정 및 학습 파라미터 최적화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

#CPU 측정

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU 사용량 및 속도 측정 함수
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # 시작 시간 기록
    start_cpu = psutil.cpu_percent(interval=None)  # 시작 CPU 사용량

    train_function()  # 학습 함수 호출

    end_time = time.time()  # 종료 시간 기록
    end_cpu = psutil.cpu_percent(interval=None)  # 종료 CPU 사용량

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

Cleaning Special Characters: 100%|██████████| 2000/2000 [00:00<00:00, 12149.25it/s]


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # 에포크 수 고정
    per_device_train_batch_size=64,
    learning_rate=5.0e-05,  # 최적화된 학습률
    weight_decay=0.01,  # 최적화된 weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",  # AdamW fused 적용 (PyTorch 2.0 이상)
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # 가장 좋은 모델을 accuracy 기준으로 선택
    greater_is_better=True,  # accuracy가 높은 모델을 저장
    save_total_limit=2,  # 저장할 모델 개수 제한
    report_to="none"
)

# 최적의 파라미터로 Trainer 구성
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 전체 학습 데이터로 평가
    compute_metrics=compute_metrics
)

# CPU 사용량 및 속도 측정
measure_cpu_usage_and_time(trainer7.train)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.057500,1.535471,0.340835
2,0.040500,1.852818,0.358721
3,0.022800,2.091974,0.460274
4,0.028200,2.062333,0.359213
5,0.014400,2.666509,0.441683
6,0.000600,3.218238,0.378111
7,0.010800,2.991965,0.355178
8,0.013100,3.002748,0.381969
9,0.000500,3.343138,0.378617
10,0.012000,3.516784,0.375778


Elapsed Time: 1102.01 seconds
Average CPU Usage: 15.20%


In [47]:
# 모델 저장
#joblib.dump(trainer7, '/content/gdrive/MyDrive/NSMC2_0.02_original.pkl')

#print("모델이 저장되었습니다.")

In [48]:
# 8. 저장된 모델로 테스트 데이터 예측 및 결과 저장
# 저장된 모델 불러오기

#loaded_model = joblib.load('/content/gdrive/MyDrive/NSMC2_0.02_original.pkl')

# 불러온 모델을 사용하여 예측 수행
predictions7 = trainer7.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]

a=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

selected_columns7['y_true']=pd.DataFrame(a)
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7.to_csv('/content/nsmc3_0.02_mlm_prediction.csv', encoding='utf-8-sig')



In [49]:
from sklearn.metrics import f1_score
import pandas as pd

df=pd.read_csv('/content/nsmc3_0.02_mlm_prediction.csv', encoding='utf-8-sig')

from sklearn.metrics import classification_report, f1_score

# y_true와 y_pred를 추출
y_true = df['y_true']
y_pred = df['y_pred']

# classification_report를 사용하여 각 클래스별 F1-score 출력
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.9485    0.1290    0.2271      1000
           1     0.5327    0.9930    0.6934      1000

    accuracy                         0.5610      2000
   macro avg     0.7406    0.5610    0.4603      2000
weighted avg     0.7406    0.5610    0.4603      2000

